In [1]:
!pip install opencv-python

You should consider upgrading via the 'c:\users\user\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


In [4]:
!pip install -U flask-cors

Requirement already up-to-date: flask-cors in c:\users\user\appdata\local\programs\python\python37\lib\site-packages (3.0.10)


You should consider upgrading via the 'c:\users\user\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


In [5]:
!pip install mediapipe

You should consider upgrading via the 'c:\users\user\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


In [6]:
!pip install simplejson

You should consider upgrading via the 'c:\users\user\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


In [7]:
###### 거리 구하는 함수
def euclidean_distance(pt1, pt2):
    distance = 0
    for i in range(len(pt1)):
        distance += (pt1[i] - pt2[i]) ** 2
    return distance ** 0.5

In [8]:
import math
pi = math.pi
# 구면 좌표로 변환하는 함수
def spherical(x, y, z):
    r= np.sqrt(x**2+y**2+z**2)
    theta = math.acos(y/r)/pi*180
    if x == 0 :
        phi = 0
    else :
        phi = math.atan2(x,z)/pi*180   
    return r, theta, phi

In [1]:
import numpy as np
import math
import time

In [3]:
import mediapipe as mp

--------- 체스트 플라이 정면 ---------

In [4]:
import cv2
import mediapipe as mp
import numpy as np
import math
import time
pi = math.pi
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

def chestflyFront(data):
    annotated_image=None
    dis = 0
    cnt = 0
    g_cnt = 0
    state = None
    feedback = None
    feedback_l = None
    feedback_r = None
    BG_COLOR = (192, 192, 192) # gray
    with mp_pose.Pose(
        static_image_mode=True,
        model_complexity=2,
        enable_segmentation=True,
        min_detection_confidence=0.5) as pose:
        for idx, file in enumerate([data]):
            ##### 이미지로 테스트할 때는 이렇게
            #image = cv2.imread(file)
            ##### 실제 연동시에는 이렇게
            image = file
            image_height, image_width, _ = image.shape
            # Convert the BGR image to RGB before processing.
            results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            
            if not results.pose_world_landmarks:
                continue

            try :
                landmarks_3d = results.pose_world_landmarks.landmark  
                l_t_x = landmarks_3d[mp_pose.PoseLandmark.LEFT_THUMB.value].x*100
                l_t_y = landmarks_3d[mp_pose.PoseLandmark.LEFT_THUMB.value].y*100
                l_t_z = landmarks_3d[mp_pose.PoseLandmark.LEFT_THUMB.value].z*100           
                left_thumb = [l_t_x, l_t_y, l_t_z]
                r_t_x = landmarks_3d[mp_pose.PoseLandmark.RIGHT_THUMB.value].x*100
                r_t_y = landmarks_3d[mp_pose.PoseLandmark.RIGHT_THUMB.value].y*100
                r_t_z = landmarks_3d[mp_pose.PoseLandmark.RIGHT_THUMB.value].z*100
                right_thumb = [r_t_x, r_t_y, r_t_z]
                dis = euclidean_distance(left_thumb, right_thumb)


                ##### 팔의 위치
                if l_t_z>-13 or r_t_z>-13 :
                    feedback = None
                    if -44.5<=l_t_y and -44.5<=r_t_y :
                        feedback = "GREAT!!"
                        #cv2.rectangle(image, (300, 400), (700, 500), (211, 211, 211), -1)    
                        cv2.putText(image, feedback, (int(0.4*w), int(0.98*h)), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 0), 3, cv2.LINE_AA)
                    if l_t_y < -44.5 :
                        feedback_l = "Down left" ###### 화면 반전으로 왼쪽, 오른쪽 바뀜
                        feedback = None
                    else :
                        feedback_l = None         
                    if r_t_y < -44.5 :
                        feedback_r = "Down right"
                        feedback = None
                    else :
                        feedback_r = None
                if dis > 110 :
                    state = "PUSH"
##                if dis < 25 and state == 'PUSH':
                if dis < 25:
                    state = "PULL"
                    cnt = 1
                    print("운동 횟수 : ", cnt)
                    if feedback=='GREAT!!':
                        g_cnt= 1
                        print("정자세 횟수 : ", cnt)

            except:
                pass
            
            annotated_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).copy()
            # Draw pose landmarks on the image.
            mp_drawing.draw_landmarks(
                annotated_image,
                results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())

            cv2.imwrite('./result_test' + str(idx) + '.png', annotated_image)
            
            #return annotated_image
    return annotated_image, cnt, g_cnt, state, feedback, feedback_l, feedback_r, dis

# # For static images:
# IMAGE_FILES = []
# file = "./test.png"
# IMAGE_FILES.append(file)
# print(type(IMAGE_FILES))
# #imageConvert(IMAGE_FILES)


--------- 시티드 로우 측면 ---------

In [5]:
import cv2
import mediapipe as mp
import numpy as np
import math
import time
pi = math.pi
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

def seatedrowSide(data):
    annotated_image=None
    cnt = 0
    g_cnt = 0
    state = None
    feedback = None

    BG_COLOR = (192, 192, 192) # gray
    with mp_pose.Pose(
        static_image_mode=True,
        model_complexity=2,
        enable_segmentation=True,
        min_detection_confidence=0.5) as pose:
        for idx, file in enumerate([data]):
            ##### 이미지로 테스트할 때는 이렇게
            #image = cv2.imread(file)
            ##### 실제 연동시에는 이렇게
            image = file
            image_height, image_width, _ = image.shape
            # Convert the BGR image to RGB before processing.
            results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            
            if not results.pose_world_landmarks:
                continue

            try :
                landmarks = results.pose_world_landmarks.landmark
                ##### 추출하고자 하는 좌표 추출
                ####################  왼쪽 어깨
                r_shou = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                                landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y,
                                landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].z ]
                r_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                               landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y,
                               landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].z]
                r_eye = [landmarks[mp_pose.PoseLandmark.RIGHT_EYE.value].x,
                               landmarks[mp_pose.PoseLandmark.RIGHT_EYE.value].y,
                               landmarks[mp_pose.PoseLandmark.RIGHT_EYE.value].z]
                r_ear = [landmarks[mp_pose.PoseLandmark.RIGHT_EAR.value].x,
                               landmarks[mp_pose.PoseLandmark.RIGHT_EAR.value].y,
                               landmarks[mp_pose.PoseLandmark.RIGHT_EAR.value].z]
                r_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                               landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y,
                               landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].z]
                r_shou_x = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].x*100
                r_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                               landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y,
                               landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].z]
                dis = euclidean_distance(r_shou, r_wrist)*100
                print(dis)

                angle = calculateAngle(r_eye, r_shou, r_hip)
                print(angle)
                angle_c = calculateAngle(r_shou, r_hip, r_knee)

                ##### 팔의 각도 체크 & 횟수 카운트
                if dis > 45 :
                    state = "down"
                    stage = "down"
                    feedback = ""
                if dis < 34 and state == 'down':
                    state = "up"
                    cnt=1
                    print(counter)


                if angle < 105 :
                    feedback = "keep your head up"
                if angle > 170 :
                    feedback = "keep your head down"
                if 100 < angle_c < 120 and stage == 'down' :
                    stage = 'up'
                    feedback = "great"
                if angle_c > 130 :
                    feedback = "fix your posture"
                    if feedback=='great':
                        g_cnt=1

            except:
                pass
            
            annotated_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).copy()
            # Draw pose landmarks on the image.
            mp_drawing.draw_landmarks(
                annotated_image,
                results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())

            cv2.imwrite('./result_test' + str(idx) + '.png', annotated_image)
            
            #return annotated_image
    return annotated_image, cnt, g_cnt, state, feedback

# # For static images:
# IMAGE_FILES = []
# file = "./test.png"
# IMAGE_FILES.append(file)
# print(type(IMAGE_FILES))
# #imageConvert(IMAGE_FILES)


--------- 시티드 로우 후면 ---------

In [6]:
import cv2
import mediapipe as mp
import numpy as np
import math
import time
pi = math.pi
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

def seatedrowBack(data):
    annotated_image=None
    cnt = 0
    g_cnt = 0
    state = None
    feedback = None
    feedback_l = None
    feedback_r = None
    BG_COLOR = (192, 192, 192) # gray
    with mp_pose.Pose(
        static_image_mode=True,
        model_complexity=2,
        enable_segmentation=True,
        min_detection_confidence=0.5) as pose:
        for idx, file in enumerate([data]):
            ##### 이미지로 테스트할 때는 이렇게
            #image = cv2.imread(file)
            ##### 실제 연동시에는 이렇게
            image = file
            image_height, image_width, _ = image.shape
            # Convert the BGR image to RGB before processing.
            results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            
            if not results.pose_world_landmarks:
                continue

            try :
                landmarks_3d = results.pose_world_landmarks.landmark  
                ##### 추출하고자 하는 좌표 추출
                ####################  왼쪽 어깨
                sh_l_x = landmarks_3d[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x*100
                sh_l_y = landmarks_3d[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y*100
                sh_l_z = landmarks_3d[mp_pose.PoseLandmark.LEFT_SHOULDER.value].z*100
                ####################  왼쪽 팔꿈치
                el_l_x = landmarks_3d[mp_pose.PoseLandmark.LEFT_ELBOW.value].x*100
                el_l_y = landmarks_3d[mp_pose.PoseLandmark.LEFT_ELBOW.value].y*100
                el_l_z = landmarks_3d[mp_pose.PoseLandmark.LEFT_ELBOW.value].z*100
                ####################  오른쪽 어깨
                sh_r_x = landmarks_3d[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x*100
                sh_r_y = landmarks_3d[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y*100
                sh_r_z = landmarks_3d[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].z*100
                ####################  오른쪽 팔꿈치
                el_r_x = landmarks_3d[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x*100
                el_r_y = landmarks_3d[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y*100
                el_r_z = landmarks_3d[mp_pose.PoseLandmark.RIGHT_ELBOW.value].z*100            
                ##### 어깨를 중심으로 만들기
                ################ 내가 원하는 좌표(팔꿈치) - 기준이 되고 싶은 좌표(어깨)
                el_l_x, el_l_y, el_l_z = el_l_x-sh_l_x, el_l_y-sh_l_y, el_l_z-sh_l_z
                el_r_x, el_r_y, el_r_z = el_r_x-sh_r_x, el_r_y-sh_r_y, el_r_z-sh_r_z
                ##### 구면좌표로 변환
                l_r, l_theta, l_phi = spherical(el_l_x, el_l_y, el_l_z)
                r_r, r_theta, r_phi = spherical(el_r_x, el_r_y, el_r_z)


                ##### 팔의 각도 체크 & 횟수 카운트
                if l_theta > 50 :
                    feedback_l = "Put your left elbows down"
                else :
                    feedback_l = ""


                if r_theta > 50 :
                    feedback_r = "Put your right elbows down"
                else :
                    feedback_r = ""


                if l_phi > 8 :
                    state = "down"
                    feedback=""
                if l_phi < -42 and stage == "down":
                    state = "up"
                    counter = 1
                    feedback=""
                    if l_theta <= 49  and r_theta <= 49 :
                        feedback="great"
                        if feedback=="great":
                            g_cnt=1
                    

            except:
                pass
            
            annotated_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).copy()
            # Draw pose landmarks on the image.
            mp_drawing.draw_landmarks(
                annotated_image,
                results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())

            cv2.imwrite('./result_test' + str(idx) + '.png', annotated_image)
            
            #return annotated_image
    return annotated_image, cnt, g_cnt, state, feedback, feedback_l, feedback_r

# # For static images:
# IMAGE_FILES = []
# file = "./test.png"
# IMAGE_FILES.append(file)
# print(type(IMAGE_FILES))
# #imageConvert(IMAGE_FILES)


--------- 덤벨 숄더 프레스 정면 ---------

In [7]:
import cv2
import mediapipe as mp
import numpy as np
import math
import time
pi = math.pi
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

def dumbbellFront(data):
    annotated_image=None
    cnt = 0
    g_cnt = 0
    state = None
    feedback = None
    feedback_l = None
    feedback_r = None
    BG_COLOR = (192, 192, 192) # gray
    with mp_pose.Pose(
        static_image_mode=True,
        model_complexity=2,
        enable_segmentation=True,
        min_detection_confidence=0.5) as pose:
        for idx, file in enumerate([data]):
            ##### 이미지로 테스트할 때는 이렇게
            #image = cv2.imread(file)
            ##### 실제 연동시에는 이렇게
            image = file
            image_height, image_width, _ = image.shape
            # Convert the BGR image to RGB before processing.
            results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            
            if not results.pose_world_landmarks:
                continue

            try :
                landmarks_3d = results.pose_world_landmarks.landmark  
                ##### 추출하고자 하는 좌표 추출
                ####################  왼쪽 어깨
                sh_l_x = landmarks_3d[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x*100
                sh_l_y = landmarks_3d[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y*100
                sh_l_z = landmarks_3d[mp_pose.PoseLandmark.LEFT_SHOULDER.value].z*100
                ####################  왼쪽 팔꿈치
                el_l_x = landmarks_3d[mp_pose.PoseLandmark.LEFT_ELBOW.value].x*100
                el_l_y = landmarks_3d[mp_pose.PoseLandmark.LEFT_ELBOW.value].y*100
                el_l_z = landmarks_3d[mp_pose.PoseLandmark.LEFT_ELBOW.value].z*100
                ####################  오른쪽 어깨
                sh_r_x = landmarks_3d[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x*100
                sh_r_y = landmarks_3d[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y*100
                sh_r_z = landmarks_3d[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].z*100
                ####################  오른쪽 팔꿈치
                el_r_x = landmarks_3d[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x*100
                el_r_y = landmarks_3d[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y*100
                el_r_z = landmarks_3d[mp_pose.PoseLandmark.RIGHT_ELBOW.value].z*100            
                ##### 어깨를 중심으로 만들기
                ################ 내가 원하는 좌표(팔꿈치) - 기준이 되고 싶은 좌표(어깨)
                el_l_x, el_l_y, el_l_z = el_l_x-sh_l_x, el_l_y-sh_l_y, el_l_z-sh_l_z
                el_r_x, el_r_y, el_r_z = el_r_x-sh_r_x, el_r_y-sh_r_y, el_r_z-sh_r_z
                ##### 구면좌표로 변환
                l_r, l_theta, l_phi = spherical(el_l_x, el_l_y, el_l_z)
                r_r, r_theta, r_phi = spherical(el_r_x, el_r_y, el_r_z)


                
                ##### 팔의 각도 체크 & 횟수 카운트
                if l_theta < 78 :
                    feedback_l = "Raise right!" ###### 화면 반전으로 왼쪽, 오른쪽 바뀜
                    feedback = None
                    stateL = "gg"
                else :
                    feedback_l = None
                if r_theta < 78 :
                    feedback_r = "Raise left!"
                    feedback = None
                    stateR = "gg"
                else :
                    feedback_r = None
                if 78<=l_theta and l_theta<=100 and 78<=r_theta and r_theta<=100 and stateL==None and stateR==None :
                    feedback = "GREAT!!"
                    state = "DOWN"
                    stateL = None
                    stateR = None
                if 145<l_theta and 145<r_theta and state == "DOWN":
                    state = " UP"
                    cnt =1
                    if feedback=='GREAT!!':
                        g_cnt =1
                if l_theta<95 or r_theta<95 :
                    feedback = None

            except:
                pass
            
            annotated_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).copy()
            # Draw pose landmarks on the image.
            mp_drawing.draw_landmarks(
                annotated_image,
                results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())

            cv2.imwrite('./result_test' + str(idx) + '.png', annotated_image)
            
            #return annotated_image
    return annotated_image, cnt, g_cnt, state, feedback, feedback_l, feedback_r

# # For static images:
# IMAGE_FILES = []
# file = "./test.png"
# IMAGE_FILES.append(file)
# print(type(IMAGE_FILES))
# #imageConvert(IMAGE_FILES)


In [9]:
from PIL import Image # pip install pillow 설치
from flask import Flask, Response, request, jsonify
from flask_cors import CORS
import numpy as np
import base64
import cv2
import time
import mediapipe as mp
import json
import simplejson as jsonn

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

app = Flask(__name__)
CORS(app)

count_dict = {}
resp_data = {}


@app.route('/sendFrame3', methods=["GET", "POST"])
# 체스트 플라이 정면
def sendFrame1():
#    display(request.args)
    exercise_id = request.args['id']
    print("exercise_id : ", exercise_id)
    byteData = request.get_data()
    frame = Image.frombuffer('RGBA', (320,240), byteData)
    numpy_image=np.array(frame)
    resp_data = {}
    result_image = chestflyFront(numpy_image)[0] ###### 넘파이 이미지에 함수를 사용해서 뭔가 결과 내기
    cnt = chestflyFront(numpy_image)[1]
    g_cnt = chestflyFront(numpy_image)[2]
    result_state = chestflyFront(numpy_image)[3]
    result_feedback = chestflyFront(numpy_image)[4]
    result_feedback_l = chestflyFront(numpy_image)[5]
    result_feedback_r = chestflyFront(numpy_image)[6]
    result_dis = chestflyFront(numpy_image)[7]
    print(result_dis)
    
    if exercise_id in count_dict:
        prev_count = count_dict[exercise_id] 
        if cnt == 1:
            cur_count = prev_count + 1
        else :
            cur_count = prev_count
        count_dict[exercise_id] = cur_count
        resp_data['result_cnt'] = cur_count
    else:
        count_dict[exercise_id] = 0
    #### g_카운트 세기
    if exercise_id+"_g" in count_dict:
        prev_count_g = count_dict[exercise_id+"_g"]        
        if g_cnt == 1:
            cur_count_g = prev_count_g + 1
        else :
            cur_count_g = prev_count_g
        count_dict[exercise_id+"_g"] = cur_count_g
        resp_data['result_g_cnt'] = cur_count_g
    else:
        resp_data['result_cnt_g'] = 0
        count_dict[exercise_id+"_g"] = 0
        
    if result_image is None :
        result_image = numpy_image
    ret, buffer = cv2.imencode('.jpg', result_image)
    
    jpg_as_text = base64.b64encode(buffer)
    
    resp_data = {
        'id': exercise_id,
        'result_image' : jpg_as_text,
        'result_cnt' : cur_count,
        'result_g_cnt' : cur_count_g,
        'result_state' : result_state,
        'result_feedback' : result_feedback,
        'result_feedback_l' : result_feedback_l,
        'result_feedback_r' : result_feedback_r,
        'result_dis': result_dis
    }
    ##print(resp_data)
    return jsonn.dumps(resp_data)


# 시티드 로우 측면
def sendFrame2():
#    display(request.args)
    exercise_id = request.args['id']
    print("exercise_id : ", exercise_id)
    byteData = request.get_data()
    frame = Image.frombuffer('RGBA', (320,240), byteData)
    numpy_image=np.array(frame)
    resp_data = {}
    result_image = seatedrowSide(numpy_image)[0] ###### 넘파이 이미지에 함수를 사용해서 뭔가 결과 내기
    cnt = seatedrowSide(numpy_image)[1]
    g_cnt = seatedrowSide(numpy_image)[2]
    result_state = seatedrowSide(numpy_image)[3]
    result_feedback = seatedrowSide(numpy_image)[4]
    #### 카운트 세기
    if exercise_id in count_dict:
        prev_count = count_dict[exercise_id] 
        if cnt == 1:
            cur_count = prev_count + 1
        else :
            cur_count = prev_count
        count_dict[exercise_id] = cur_count
        resp_data['result_cnt'] = cur_count
    else:
        resp_data['result_cnt'] = 0
        count_dict[exercise_id] = 0
    #### g_카운트 세기
    if exercise_id+"_g" in count_dict:
        prev_count_g = count_dict[exercise_id+"_g"]        
        if g_cnt == 1:
            cur_count_g = prev_count_g + 1
        else :
            cur_count_g = prev_count_g
        count_dict[exercise_id+"_g"] = cur_count_g
        resp_data['result_g_cnt'] = cur_count_g
    else:
        resp_data['result_cnt_g'] = 0
        count_dict[exercise_id+"_g"] = 0
        
    if result_image is None :
        result_image = numpy_image
    ret, buffer = cv2.imencode('.jpg', result_image)
    
    jpg_as_text = base64.b64encode(buffer)
    
    resp_data = {
        'id': exercise_id,
        'result_image' : jpg_as_text,
        'result_cnt' : cur_count,
        'result_g_cnt' : cur_count_g,
        'result_state' : result_state,
        'result_feedback' : result_feedback
    }
    ##print(resp_data)
    return jsonn.dumps(resp_data)


# 시티드로우 후면
@app.route('/sendFrame2', methods=["GET", "POST"])
def sendFrame3():
#    display(request.args)
    exercise_id = request.args['id']
    print("exercise_id : ", exercise_id)
    byteData = request.get_data()
    frame = Image.frombuffer('RGBA', (320,240), byteData)
    numpy_image=np.array(frame)
    resp_data = {}
    result_image = seatedrowBack(numpy_image)[0] ###### 넘파이 이미지에 함수를 사용해서 뭔가 결과 내기
    cnt = seatedrowBack(numpy_image)[1]
    g_cnt = seatedrowBack(numpy_image)[2]
    result_state = seatedrowBack(numpy_image)[3]
    result_feedback = seatedrowBack(numpy_image)[4]
    result_feedback_l = seatedrowBack(numpy_image)[5]
    result_feedback_r = seatedrowBack(numpy_image)[6]
    #### 카운트 세기
    if exercise_id in count_dict:
        prev_count = count_dict[exercise_id] 
        if cnt == 1:
            cur_count = prev_count + 1
        else :
            cur_count = prev_count
        count_dict[exercise_id] = cur_count
        resp_data['result_cnt'] = cur_count
    else:
        resp_data['result_cnt'] = 0
        count_dict[exercise_id] = 0
    #### g_카운트 세기
    if exercise_id+"_g" in count_dict:
        prev_count_g = count_dict[exercise_id+"_g"]        
        if g_cnt == 1:
            cur_count_g = prev_count_g + 1
        else :
            cur_count_g = prev_count_g
        count_dict[exercise_id+"_g"] = cur_count_g
        resp_data['result_g_cnt'] = cur_count_g
    else:
        resp_data['result_cnt_g'] = 0
        count_dict[exercise_id+"_g"] = 0
        
    if result_image is None :
        result_image = numpy_image
    ret, buffer = cv2.imencode('.jpg', result_image)
    
    jpg_as_text = base64.b64encode(buffer)
    
    resp_data = {
        'id': exercise_id,
        'result_image' : jpg_as_text,
        'result_cnt' : cur_count,
        'result_g_cnt' : cur_count_g,
        'result_state' : result_state,
        'result_feedback' : result_feedback,
        'result_feedback_l' : result_feedback_l,
        'result_feedback_r' : result_feedback_r
    }
    ##print(resp_data)
    return jsonn.dumps(resp_data)


# 덤벨 숄더 프레스 #######
@app.route('/sendFrame333', methods=["GET", "POST"])
def sendFrame4():
#    display(request.args)
    exercise_id = request.args['id']
    print("exercise_id : ", exercise_id)
    byteData = request.get_data()
    frame = Image.frombuffer('RGBA', (320,240), byteData)
    numpy_image=np.array(frame)
    resp_data = {}
    result_image = dumbbellFront(numpy_image)[0] ###### 넘파이 이미지에 함수를 사용해서 뭔가 결과 내기
    cnt = dumbbellFront(numpy_image)[1]
    g_cnt = dumbbellFront(numpy_image)[2]
    result_state = dumbbellFront(numpy_image)[3]
    result_feedback = dumbbellFront(numpy_image)[4]
    result_feedback_l = dumbbellFront(numpy_image)[5]
    result_feedback_r = dumbbellFront(numpy_image)[6]
    #### 카운트 세기
    if exercise_id in count_dict:
        prev_count = count_dict[exercise_id] 
        if cnt == 1:
            cur_count = prev_count + 1
        else :
            cur_count = prev_count
        count_dict[exercise_id] = cur_count
        resp_data['result_cnt'] = cur_count
    else:
        resp_data['result_cnt'] = 0
        count_dict[exercise_id] = 0
    #### g_카운트 세기
    if exercise_id+"_g" in count_dict:
        prev_count_g = count_dict[exercise_id+"_g"]        
        if g_cnt == 1:
            cur_count_g = prev_count_g + 1
        else :
            cur_count_g = prev_count_g
        count_dict[exercise_id+"_g"] = cur_count_g
        resp_data['result_g_cnt'] = cur_count_g
    else:
        resp_data['result_cnt_g'] = 0
        count_dict[exercise_id+"_g"] = 0
        
    if result_image is None :
        result_image = numpy_image
    ret, buffer = cv2.imencode('.jpg', result_image)
    
    jpg_as_text = base64.b64encode(buffer)
    
    resp_data = {
        'id': exercise_id,
        'result_image' : jpg_as_text,
        'result_cnt' : cur_count,
        'result_g_cnt' : cur_count_g,
        'result_state' : result_state,
        'result_feedback' : result_feedback,
        'result_feedback_l' : result_feedback_l,
        'result_feedback_r' : result_feedback_r
    }
    ##print(resp_data)
    return jsonn.dumps(resp_data)


if __name__ == '__main__':
    app.run(host="localhost", port="5000")

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


액세스 권한에 의해 숨겨진 소켓에 액세스를 시도했습니다
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "c:\Users\user\AppData\Local\Programs\Python\Python37\lib\site-packages\werkzeug\serving.py", line 908, in prepare_socket
    s.bind(server_address)
OSError: [WinError 10013] 액세스 권한에 의해 숨겨진 소켓에 액세스를 시도했습니다

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\user\AppData\Local\Programs\Python\Python37\lib\site-packages\IPython\core\interactiveshell.py", line 3552, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\user\AppData\Local\Temp\ipykernel_22304\4055752069.py", line 269, in <module>
    app.run(host="localhost", port="5000")
  File "c:\Users\user\AppData\Local\Programs\Python\Python37\lib\site-packages\flask\app.py", line 920, in run
    run_simple(t.cast(str, host), port, self, **options)
  File "c:\Users\user\AppData\Local\Programs\Python\Python37\lib\site-packages\werkzeug\serving.py", line 1059, in run_simple
    s = prepare_socket(h

TypeError: object of type 'NoneType' has no len()